In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SequentialFeatureSelector


In [2]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [3]:
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import LogisticRegression


#log_model = LogisticRegression(solver='lbfgs')
#DT=DecisionTreeClassifier(criterion='gini',max_features='sqrt',splitter='best',random_state=0)
RF=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
sfs = SequentialFeatureSelector(RF, n_features_to_select=10, direction='forward',scoring='accuracy', cv=5)

sfs = sfs.fit(indep_X,dep_Y)
sfs
    


selected_features = sfs.get_support(indices=True)
SFSlist = indep_X.columns[selected_features].tolist()

print("Selected feature indices:", selected_features)
print("Selected feature names:", SFSlist)
        
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        sc.fit_transform(X_train)
        #X_test = sc.transform(X_test)
        return X_train, X_test, y_train, y_test,sc

    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm


Selected feature indices: [ 2  3  5 10 11 13 14 15 17 22]
Selected feature names: ['al', 'su', 'bu', 'pcv', 'wc', 'sg_b', 'sg_c', 'sg_d', 'rbc_normal', 'dm_yes']


In [4]:
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [5]:
df2['rbc_normal']

0      1
1      1
2      1
3      1
4      1
      ..
394    1
395    1
396    1
397    1
398    1
Name: rbc_normal, Length: 399, dtype: uint8

In [6]:
def logistic(X_train,y_train,X_test):
    
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    

    

def sfs_classification(acclog):
   
    sfs_dataframe = pd.DataFrame(index=['RF'],
                                 columns=['Logistic'])

   
    for number, model_name in enumerate(sfs_dataframe.index):
        sfs_dataframe.loc[model_name, 'Logistic'] = acclog[number]

    return sfs_dataframe

In [7]:
#SFSlist=sfsFeature(indep_X,dep_Y,4)       

acclog=[]


In [8]:
for i in SFSlist:   
    
    feature = indep_X[[i]]
    X_train, X_test, y_train, y_test,sc=split_scalar(feature,dep_Y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    acclog.append(Accuracy)
    
    
    
result=sfs_classification(acclog)

C:\sabeen\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\sabeen\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\sabeen\anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\sabeen\anaconda3\envs\aiml\lib\site-p

In [9]:
result
#4

,Logistic
RF,0.8


In [10]:
import pickle as pkl
filename="finalized_model_Random.sav"

In [11]:
pickle.dump(classifier,open(filename,'wb'))

In [12]:
filenamex="sc.pkl"

In [13]:
pickle.dump(sc,open(filenamex,'wb'))

In [14]:
sc=pickle.load(open("sc.pkl",'rb'))

In [15]:
preinput = sc.transform([[3, 0.0, 51, 38.86890244, 10300, 0, 0, 1, 0, 0]])
preinput

C:\sabeen\anaconda3\envs\aiml\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


ValueError: X has 10 features, but StandardScaler is expecting 1 features as input.

In [ ]:
loaded_model=pickle.load(open("finalized_model_Random.sav",'rb')) 
result = loaded_model.predict(preinput)

In [ ]:
result